# FYP 2022-2026: Preprocessing + Modeling

In [ ]:
%reload_kedro

In [ ]:
import pandas as pd

In [ ]:
from fyp_analysis.pipelines.modeling.predict.utils import get_possible_endog_variables
from fyp_analysis.pipelines.modeling.predict.core import (
    run_possible_models,
    fit_var_model,
)
from fyp_analysis.pipelines.modeling.predict.viz import plot_fit

from fyp_analysis.pipelines.data_processing.preprocess.core import PreprocessPipeline
from fyp_analysis.extras.datasets.cbo import load_cbo_data

In [ ]:
DATA = catalog
PARAMS = context.params

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
DATA.list()

In [ ]:
min_year = PARAMS['min_feature_year']
cbo_forecast_date = PARAMS['cbo_forecast_date']
max_fit_date = PARAMS['max_fit_date']
plan_start_year = PARAMS['plan_start_year']

In [ ]:
C = DATA.load("scaled_feature_correlations")
G = DATA.load("scaled_feature_correlations")

In [ ]:
unscaled_features = DATA.load("final_unscaled_features")

In [ ]:
guide = DATA.load("stationary_guide")
preprocess = PreprocessPipeline(guide)

In [ ]:
cbo_data = load_cbo_data(date=cbo_forecast_date)
cbo_columns = cbo_data.columns.tolist()

# Forecasts

In [ ]:
all_bestfit_params = {}
all_comparisons = {}
all_tax_bases = {}

## Wage

In [ ]:
C['D.Ln.WageBase'].sort_values().head(n=10)

In [ ]:
C['D.Ln.WageBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "WageBase", alpha=0.05)

In [ ]:
wage_fits = run_possible_models(
    unscaled_features,
    preprocess,
    main_endog="WageBase",
    other_endog=[
        "InitialClaimsPA",
        "NonfarmEmployeesPhilly",
        "NonfarmEmployeesPhillyMSA",
        "UnemploymentPhillyMSA",
        "UnemploymentPhilly",
        "PrimeEPOP",
    ],
    orders=[2, 3, 4, 5, 6, 7, 8],
    grangers=G,
    max_fit_date="2019-12-31",
    cbo_columns=cbo_columns,
    alpha=0.05,
    max_exog=4,
    split_year=2014,
    max_other_endog=2,
    model_quarters=True
)

In [ ]:
best_params = wage_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    unscaled_features,
    preprocess,
    plan_start_year=plan_start_year,
    max_fit_date="2019-12-31",
    cbo_data=cbo_data,
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    model_quarters=best_params['model_quarters']
)
print(result.aic)

In [ ]:
result.summary()

In [ ]:
fig = plot_fit(forecast, "WageBase");

### FYP Comparison

In [ ]:
# Calculate avg forecast
forecast_wage_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), wage_fits, "WageBase"
)

all_bestfit_params["Wage"] = fits

In [ ]:
(forecast_wage_base.diff() / forecast_wage_base.shift()).tail(n=7)

In [ ]:
wageTax = TAXES['Wage'](add_net_accrual=True)

In [ ]:
all_comparisons["Wage"] = (
    wageTax.get_mayor_comparison(forecast_wage_base).loc[2020:]
    .assign(WageRevenueBudget=lambda df: df.WageRevenueBudget.fillna(df.WageRevenue))
    .rename(
        columns={"WageRevenueBudget": "Five Year Plan", "WageRevenue": "Controller"}
    )
)

all_tax_bases['Wage'] = forecast_wage_base

In [ ]:
all_comparisons['Wage']/1e6

In [ ]:
wageTax.plot_summary(forecast_wage_base, figsize=(6.4, 5))
plt.savefig(RESULTS_DIR / "summary" / "figures" / "wage_comparison.png")

## Sales

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.Ln.SalesBase'].sort_values().head(n=10)

In [ ]:
C['D.Ln.SalesBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "SalesBase", alpha=0.1)

In [ ]:
sales_fits = run_possible_models(
    get_selected_features(data, min_year),
    "SalesBase",
    ["ConsumerConfidence", "CarSales", "CorporateProfits",],
    [2, 3, 4, 5, 6],
    G,
    alpha=0.1,
    max_exog=4,
    split_year=2014,
    max_other_endog=2,
)

In [ ]:
best_params = sales_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)
print(result.aic)

In [ ]:
result.summary()

In [ ]:
fig = plot_fit(forecast, "SalesBase");

### FYP Comparison

In [ ]:
# Get avg forecasts
forecast_sales_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), sales_fits, "SalesBase", max_fits=1
)

# Save
all_bestfit_params['Sales'] = fits

In [ ]:
(forecast_sales_base.diff() / forecast_sales_base.shift()).tail(n=7)

In [ ]:
salesTax = TAXES['Sales'](add_net_accrual=True)

In [ ]:
all_comparisons["Sales"] = (
    salesTax.get_mayor_comparison(forecast_sales_base).loc[2020:]
    .assign(SalesRevenueBudget=lambda df: df.SalesRevenueBudget.fillna(df.SalesRevenue))
    .rename(
        columns={"SalesRevenueBudget": "Five Year Plan", "SalesRevenue": "Controller"}
    )
)

all_tax_bases['Sales'] = forecast_sales_base

In [ ]:
all_comparisons['Sales']/1e6

In [ ]:
salesTax.plot_summary(forecast_sales_base, figsize=(6.4, 5))
plt.savefig(RESULTS_DIR / "summary" / "figures" / "sales_comparison.png")

## BIRT

### Gross Receipts

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.GrossReceiptsBase'].sort_values().head(n=10)

In [ ]:
C['D.GrossReceiptsBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "GrossReceiptsBase", alpha=0.2)

In [ ]:
gross_receipts_fits = run_possible_models(
    get_selected_features(data, min_year),
    "GrossReceiptsBase",
    [
        "ConsumerConfidence",
        "CorporateProfits",
        "RealRetailFoodServiceSales",
        "CPIPhillyMSA",
        "CarSales",
        "PCEPriceIndex"
    ],
    [2, 3, 4, 5, 6],
    G,
    alpha=0.2,
    max_exog=4,
    split_year=2014,
    max_other_endog=1,
)

In [ ]:
best_params = gross_receipts_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)
print(result.aic)

In [ ]:
result.summary()

In [ ]:
plot_fit(forecast, "GrossReceiptsBase");

In [ ]:
forecast_gr_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year),
    gross_receipts_fits,
    "GrossReceiptsBase",
    max_fits=3,
)

all_bestfit_params['GrossReceipts'] = fits

In [ ]:
(forecast_gr_base.diff() / forecast_gr_base.shift()).tail(n=7)

### Net Income

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.NetIncomeBase'].sort_values().head(n=10)

In [ ]:
C['D.NetIncomeBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "NetIncomeBase", alpha=0.1)

In [ ]:
net_income_fits = run_possible_models(
    get_selected_features(data, min_year),
    "NetIncomeBase",
    [
        "ConsumerConfidence",
        "CorporateProfits",
        "UnemploymentPhillyMSA",
        "InitialClaimsPA",
        "CPIPhillyMSA",
        "SP500",
        "PCEPriceIndex",
    ],
    [2, 3, 4, 5, 6],
    G,
    alpha=0.2,
    max_exog=4,
    split_year=2014,
    max_other_endog=2,
)

In [ ]:
best_params = net_income_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)
print(result.aic)

In [ ]:
result.summary()

In [ ]:
plot_fit(forecast, "NetIncomeBase");

In [ ]:
# Get avg forecast
forecast_ni_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), net_income_fits, "NetIncomeBase", max_fits=1
)

all_bestfit_params['NetIncome'] = fits

In [ ]:
(forecast_ni_base.diff() / forecast_ni_base.shift()).tail(n=7)

### FYP Comparison

In [ ]:
BIRT = TAXES['BIRT'](add_net_accrual=True)

In [ ]:
BIRT_base = (forecast_gr_base + forecast_ni_base).rename("BIRTBase")

In [ ]:
BIRT_base.diff() / BIRT_base.shift()

In [ ]:
# Recalibrate FY22 to FY19
FY19_CALIBRATION = 0.075

# Net Income
forecast_ni_base_corr = forecast_ni_base.copy()
forecast_ni_base_corr.loc[2022:] += forecast_ni_base_corr.loc[2022] * FY19_CALIBRATION

# Gross Receipts
forecast_gr_base_corr = forecast_gr_base.copy()
forecast_gr_base_corr.loc[2022:] += forecast_gr_base_corr.loc[2022] * FY19_CALIBRATION

In [ ]:
all_comparisons["BIRT"] = (
    BIRT.get_mayor_comparison(forecast_gr_base_corr, forecast_ni_base_corr)
    .loc[2020:]
    .assign(BIRTRevenueBudget=lambda df: df.BIRTRevenueBudget.fillna(df.BIRTRevenue))
    .rename(
        columns={"BIRTRevenueBudget": "Five Year Plan", "BIRTRevenue": "Controller"}
    )
)

all_tax_bases["BIRT"] = (forecast_gr_base_corr + forecast_ni_base_corr).rename(
    "BIRTBase"
)
all_tax_bases["GrossReceipts"] = forecast_gr_base_corr
all_tax_bases["NetIncome"] = forecast_ni_base_corr

In [ ]:
all_comparisons['BIRT']/1e6

In [ ]:
BIRT.plot_summary(forecast_gr_base_corr, forecast_ni_base_corr, figsize=(6.4, 5))
plt.savefig(RESULTS_DIR / "summary" / "figures" / "birt_comparison.png")

## RTT

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.Ln.RTTBase'].sort_values().head(n=10)

In [ ]:
C['D.Ln.RTTBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "RTTBase", alpha=0.05)

In [ ]:
rtt_fits = run_possible_models(
    get_selected_features(data, min_year),
    "RTTBase",
    [
        "BuildingPermitsPhillyMSA",
        "RealDisposablePersonalIncome",
        "ConsumerConfidence",
        "NYCGasPrice"
    ],
    [1, 2, 3, 4, 5, 6],
    G,
    alpha=0.05,
    max_exog=4,
    split_year=2014,
    max_other_endog=4,
)

In [ ]:
best_params = rtt_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)
print(result.aic)

In [ ]:
result.summary()

In [ ]:
plot_fit(forecast, "RTTBase");

### FYP Comparison

In [ ]:
forecast_rtt_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), rtt_fits, "RTTBase", max_fits=3
)

all_bestfit_params["RTT"] = fits

In [ ]:
(forecast_rtt_base.diff() / forecast_rtt_base.shift()).tail(n=7)

In [ ]:
RTT = TAXES['RTT'](add_net_accrual=True)

In [ ]:
all_comparisons["RTT"] = (
    RTT.get_mayor_comparison(forecast_rtt_base)
    .loc[2020:]
    .assign(RTTRevenueBudget=lambda df: df.RTTRevenueBudget.fillna(df.RTTRevenue))
    .rename(columns={"RTTRevenueBudget": "Five Year Plan", "RTTRevenue": "Controller"})
)

all_tax_bases["RTT"] = forecast_rtt_base

In [ ]:
RTT.plot_summary(forecast_rtt_base, figsize=(6.4, 5))
plt.savefig(RESULTS_DIR / "summary" / "figures" / "rtt_comparison.png")

## Parking

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.Ln.ParkingBase'].sort_values().head(n=10)

In [ ]:
C['D.Ln.ParkingBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "ParkingBase", alpha=0.05)

In [ ]:
parking_fits = run_possible_models(
    get_selected_features(data, min_year),
    "ParkingBase",
    [
        "NonresidentialInvestment",
        "CorporateProfits",
        "NYCGasPrice",
        "NonfarmEmployeesPhilly",
        "UnemploymentPhillyMSA"
    ],
    [2, 3, 4, 5, 6],
    G,
    alpha=0.1,
    max_exog=4,
    split_year=2014,
    max_other_endog=1,
)

In [ ]:
best_params = parking_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)

In [ ]:
result.summary()

In [ ]:
plot_fit(forecast, "ParkingBase");

### FYP Comparison

In [ ]:
forecast_parking_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), parking_fits, "ParkingBase", max_fits=1
)

all_bestfit_params['Parking'] = fits

In [ ]:
(forecast_parking_base.diff() / forecast_parking_base.shift()).tail(n=7)

In [ ]:
# Recalibrate FY21
PARKING_CALIBRATION = 0.935

# Net Income
forecast_parking_base_corr = forecast_parking_base.copy()
forecast_parking_base_corr.loc[2021] *= PARKING_CALIBRATION

In [ ]:
(forecast_parking_base_corr.diff() / forecast_parking_base_corr.shift()).tail(n=7)

In [ ]:
parkingTax = TAXES['Parking'](add_net_accrual=True)

In [ ]:
all_comparisons["Parking"] = (
    parkingTax.get_mayor_comparison(forecast_parking_base_corr)
    .loc[2020:]
    .assign(
        ParkingRevenueBudget=lambda df: df.ParkingRevenueBudget.fillna(
            df.ParkingRevenue
        )
    )
    .rename(
        columns={
            "ParkingRevenueBudget": "Five Year Plan",
            "ParkingRevenue": "Controller",
        }
    )
)

all_tax_bases["Parking"] = forecast_parking_base_corr

In [ ]:
all_comparisons['Parking']/1e6

In [ ]:
parkingTax.plot_summary(forecast_parking_base_corr, figsize=(6.4, 5))
plt.savefig(RESULTS_DIR / "summary" / "figures" / "parking_comparison.png")

## Amusement

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.AmusementBase'].sort_values().head(n=10)

In [ ]:
C['D.AmusementBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "AmusementBase", alpha=0.05)

In [ ]:
amusement_fits = run_possible_models(
    get_selected_features(data, min_year),
    "AmusementBase",
    [
        "ParkingBase",
        "RealGDP",
        "RealDisposablePersonalIncome",
        "PersonalIncome",
        "PersonalSavingsRate",
        "UnemploymentPhilly",
        "HousingSupply",
        "NYCGasPrice",
    ],
    [1, 2, 3, 4, 5, 6],
    G,
    alpha=0.1,
    max_exog=4,
    split_year=2014,
    max_other_endog=3,
)

In [ ]:
best_params = amusement_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)

In [ ]:
result.summary()

In [ ]:
plot_fit(forecast, "AmusementBase");

### FYP Comparison

In [ ]:
forecast_amusement_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), amusement_fits, "AmusementBase", max_fits=1
)

all_bestfit_params['Amusement'] = fits

In [ ]:
(forecast_amusement_base.diff() / forecast_amusement_base.shift()).tail(n=7)

In [ ]:
amusementTax = TAXES['Amusement'](add_net_accrual=True)

In [ ]:
all_comparisons["Amusement"] = (
    amusementTax.get_mayor_comparison(forecast_amusement_base)
    .loc[2020:]
    .assign(
        AmusementRevenueBudget=lambda df: df.AmusementRevenueBudget.fillna(
            df.AmusementRevenue
        )
    )
    .rename(
        columns={
            "AmusementRevenueBudget": "Five Year Plan",
            "AmusementRevenue": "Controller",
        }
    )
)

all_tax_bases["Amusement"] = forecast_amusement_base

In [ ]:
amusementTax.plot_summary(forecast_amusement_base, figsize=(6.4, 5))
plt.savefig(RESULTS_DIR / "summary" / "figures" / "amusement_comparison.png")

## NPT

In [ ]:
min_year = 1996
C = corr[min_year]
G = grangers[min_year]

In [ ]:
C['D.NPTBase'].sort_values().head(n=10)

In [ ]:
C['D.NPTBase'].sort_values().tail(n=10)

In [ ]:
get_possible_endog_variables(G, "NPTBase", alpha=0.1)

In [ ]:
npt_fits = run_possible_models(
    get_selected_features(data, min_year),
    "NPTBase",
    [
        "WageBase",
        "MedianHomeValuePhilly",
        "Wage&Salaries",
        "CorporateProfits",
        "RealGDP",
        "GDPPriceIndex",
        "ContinuedClaimsPA"
    ],
    [2, 3, 4, 5, 6],
    G,
    alpha=0.1,
    max_exog=4,
    split_year=2014,
    max_other_endog=3,
)

In [ ]:
best_params = npt_fits[0]
best_params

In [ ]:
result, forecast = fit_var_model(
    get_selected_features(data, min_year),
    endog_cols=best_params["endog_cols"],
    order=best_params["order"],
    exog_cols=best_params["exog_cols"],
    exclude_2020=True,
)

In [ ]:
result.summary()

In [ ]:
plot_fit(forecast, "NPTBase");

### FYP Comparison

In [ ]:
forecast_npt_base, fits = get_avg_forecast_from_fits(
    get_selected_features(data, min_year), npt_fits, "NPTBase", max_fits=2
)

all_bestfit_params['NPT'] = fits

In [ ]:
(forecast_npt_base.diff() / forecast_npt_base.shift()).tail(n=7)

In [ ]:
NPT = TAXES['NPT'](add_net_accrual=True)

In [ ]:
all_comparisons["NPT"] = (
    NPT.get_mayor_comparison(forecast_npt_base)
    .loc[2020:]
    .assign(
        NPTRevenueBudget=lambda df: df.NPTRevenueBudget.fillna(
            df.NPTRevenue
        )
    )
    .rename(columns={"NPTRevenueBudget": "Five Year Plan", "NPTRevenue": "Controller"})
)

all_tax_bases["NPT"] = forecast_npt_base

In [ ]:
NPT.plot_summary(forecast_npt_base)
plt.savefig(RESULTS_DIR / "summary" / "figures" / "npt_comparison.png")

# Summary Comparison

## Tax Bases

In [ ]:
tax_base_output = pd.concat(
    [all_tax_bases[name] for name in all_tax_bases], axis=1
)
tax_base_output

In [ ]:
tax_base_output.loc[2020:].reset_index().to_excel(
    RESULTS_DIR / "summary" / "spreadsheets" / "projected_tax_bases.xlsx", index=False
)

## Combined Comparison to Mayor Projections

In [ ]:
combined_comparison = pd.concat(
    [all_comparisons[name].assign(name=name) for name in all_comparisons]
)
combined_comparison

In [ ]:
combined_comparison.reset_index().to_excel(
    RESULTS_DIR / "summary" / "spreadsheets" / "combined_comparison.xlsx", index=False
)

## Save Endog/Exog Variables

In [ ]:
endog = {}
exog = {}

for name in all_bestfit_params:
    endog[name] = []
    exog[name] = []
    for fit in all_bestfit_params[name]:
        endog[name] += fit['endog_cols']
        exog[name] += fit['exog_cols']
        
    endog[name] = sorted(set(endog[name]))
    exog[name] = sorted(set(exog[name]))
    
    
endog = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in endog.items()]))
exog = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in exog.items()]))

In [ ]:
with pd.ExcelWriter(
    RESULTS_DIR / "summary" / "spreadsheets" / "fit_variables.xlsx"
) as writer:

    endog.to_excel(writer, sheet_name="Endog", index=False)
    exog.to_excel(writer, sheet_name="Exog", index=False)

## Save bestfit params

In [ ]:
pickle.dump(
    all_bestfit_params,
    (RESULTS_DIR / "summary" / "model_params" / "bestfits.pickle").open("wb"),
)